# Advanced Applications of Mutate

## Map and apply

In [1]:
import pandas as pd
from dfply import *
import matplotlib.pylab as plt
%matplotlib inline

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Artists

In [3]:
artists = pd.read_csv("./data/Artists.csv")
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


# Transforming columns with the `map` and `apply` methods

Next, we will take a look at two useful `pandas Series` methods that allow us to apply very general transformations: `map` and `apply`.

## Transforming a column with `map`

`df.col.map` can be used to

* Apply a translation `dict`
* Apply a function
* Apply a `pd.Series`

In [4]:
artists.Gender.value_counts()

Male          9762
Female        2300
male            15
Non-Binary       2
Non-binary       1
female           1
Name: Gender, dtype: int64

#### `map`ping a translation `dict`

In [5]:
new_gender = {'Male':'m', 'Female':'f', 'male':'m', 'female':'f', 'Non-Binary':'nb', 'Non-binary':'nb'}
(artists
 >> select(X.Gender)
 >> mutate(new_gender = X.Gender.map(new_gender))
 >> head(9)
)

,Gender,new_gender
0,Male,m
1,Male,m
2,Male,m
3,Male,m
4,Male,m
5,Male,m
6,Male,m
7,Male,m
8,Female,f


#### Setting a default with `collections.defaultdict`

In [6]:
from collections import defaultdict

from_america = defaultdict(lambda: 'Not America')
from_america.update({'American':'America'})

#### Applying the `defaultdict`

In [7]:
(artists
 >> select(X.Nationality)
 >> mutate(from_america = X.Nationality.map(from_america))
 >> head(3)
)

,Nationality,from_america
0,American,America
1,Spanish,Not America
2,American,America


#### `map`ping a simple function

In [8]:
(artists
 >> select(X.Nationality)
 >> mutate(from_USA = X.Nationality.map(lambda n: 'USA' if n == 'American' else 'Other'))
 >> head(3)
)

,Nationality,from_USA
0,American,USA
1,Spanish,Other
2,American,USA


## Be sure to `apply` yourself!

* `df.col.apply` is used to apply any function to a column.
    * Including positional and keyword arguments
* Could literally be used to perform *any* mutation

#### Applying a unary function

In [9]:
century = lambda year_string: (int(year_string)//100)*100

(artists
 >> select(X.BeginDate)
 >> mutate(century_of_birth = X.BeginDate.apply(century))
 >> head(3)
)

,BeginDate,century_of_birth
0,1930,1900
1,1936,1900
2,1941,1900


## Using anonymous functions

* There is no need to name a `lambda`
* An embedded `lambda` is called an **anonymous function**

In [10]:
(artists
 >> select(X.EndDate)
 >> mutate(new_end_date = (X.EndDate
                           .apply(lambda y: y if int(y) > 0 else np.nan)
                           .astype('Int64')))
 >> head(2)
)

,EndDate,new_end_date
0,1992,1992
1,0,<NA>


## `apply` or `map`

* Use `map` for simple functions
* Use `apply` when adding additional arguments

#### MoMA Artwork

In [12]:
from more_dfply import fix_names

artwork = (pd.read_csv("./data/Artworks.csv")
           >> fix_names
           >> mutate(id = X.index + 1)
          )
artwork.head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Weight_kg,Width_cm,Seat_Height_cm,Duration_sec,id
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN,1
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN,2


#### Setting a positional argument

We want to apply `round(val, 1)`


In [13]:
(artwork
 >> select(X.Height_cm)
 >> mutate(rounded_height = X.Height_cm.apply(round, args=(1,)))
 >> head(3)
)

,Height_cm,rounded_height
0,48.6000,48.6
1,40.6401,40.6
2,34.3000,34.3


#### Setting a keyword argument

We want to apply `logp1(val, base=n)`

In [14]:
from math import log, e

log1p = lambda num, base=e: log(num + 1, base)
(artwork
 >> select(X.Height_cm)
 >> mutate(log10_plus_1 = X.Height_cm.apply(log1p, base = 10),
           log2_plus_1 = X.Height_cm.apply(log1p, base = 2),
           ln_plus_1 = X.Height_cm.apply(log1p, base = e))
 >> head(3)
)

,Height_cm,log10_plus_1,log2_plus_1,ln_plus_1
0,48.6000,1.695482,5.632268,3.903991
1,40.6401,1.619512,5.379902,3.729064
2,34.3000,1.547775,5.141596,3.563883


## <font color="red"> Exercise 2 </font>

An **Indicator column** for a category contains 1 for the rows that match that label and 0 otherwise.  The `exhibitions` dataframe.  Complete the following tasks.

1. Use `exhibitions.ExhibitionRole.unique()` to get a list of unique columns.
2. Use `mutate` and `map` with `defaultdict` to create an indicator column for each category (ignore missing rows).
3. Comment on the quality of your solution, especially in light of the [DRY principle](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)

#### MoMA Exhibitions

In [18]:
dat_cols = ['ExhibitionBeginDate', 'ExhibitionEndDate', 'ConstituentBeginDate' ,'ConstituentEndDate']
exhibitions = pd.read_csv('./data/MoMAExhibitions1929to1989.csv', 
                          encoding="ISO-8859-1",
                          parse_dates=dat_cols)
exhibitions.head(50)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Institution,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL
0,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Curator,Director,...,NaN,American,1902,1981,"American, 19021981",Male,109252853.0,Q711362,500241556.0,moma.org/artists/9168
1,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1839,1906,"French, 18391906",Male,39374836.0,Q35548,500004793.0,moma.org/artists/1053
2,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1848,1903,"French, 18481903",Male,27064953.0,Q37693,500011421.0,moma.org/artists/2098
3,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,Dutch,1853,1890,"Dutch, 18531890",Male,9854560.0,Q5582,500115588.0,moma.org/artists/2206
4,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1859,1891,"French, 18591891",Male,24608076.0,Q34013,500008873.0,moma.org/artists/5358
5,2724.0,2,Paintings by 19 Living Americans,"[MoMA Exh. #2, December 12-January 12, 1930]",1929-12-12,1930-01-12,2.0,moma.org/calendar/exhibitions/1912,Artist,Artist,...,NaN,American,1893,1967,"American, 18931967",Male,34533792.0,Q3349279,500015189.0,moma.org/artists/870
6,2724.0,2,Paintings by 19 Living Americans,"[MoMA Exh. #2, December 12-January 12, 1930]",1929-12-12,1930-01-12,2.0,moma.org/calendar/exhibitions/1912,Artist,Artist,...,NaN,American,1883,1935,"American, 18831935",Male,29542716.0,Q380494,500004441.0,moma.org/artists/1490
7,2724.0,2,Paintings by 19 Living Americans,"[MoMA Exh. #2, December 12-January 12, 1930]",1929-12-12,1930-01-12,2.0,moma.org/calendar/exhibitions/1912,Artist,Artist,...,NaN,American,1889,1930,"American, 18891930",Male,25491785.0,Q3402687,500124280.0,moma.org/artists/1537
8,2724.0,2,Paintings by 19 Living Americans,"[MoMA Exh. #2, December 12-January 12, 1930]",1929-12-12,1930-01-12,2.0,moma.org/calendar/exhibitions/1912,Artist,Artist,...,NaN,American,1871,1956,"American, 18711956",Male,49225327.0,Q158255,500115308.0,moma.org/artists/1832
9,2724.0,2,Paintings by 19 Living Americans,"[MoMA Exh. #2, December 12-January 12, 1930]",1929-12-12,1930-01-12,2.0,moma.org/calendar/exhibitions/1912,Artist,Artist,...,NaN,American,1868,1933,"American, 18681933",Male,61822646.0,Q19795994,500027420.0,moma.org/artists/2519


In [20]:
# Your code here
exhibitions.ExhibitionRole.unique()

array(['Curator', 'Artist', nan, 'Arranger', 'Installer',
       'Competition Judge', 'Designer', 'Preparer'], dtype=object)

In [21]:
exhibitions.ExhibitionRole.value_counts()

Artist               32537
Curator               1495
Installer              150
Competition Judge      132
Designer                66
Arranger                31
Preparer                13
Name: ExhibitionRole, dtype: int64

In [26]:
from collections import defaultdict

is_artist = defaultdict(lambda: '0')
is_artist.update({'Artist':'1'})

In [27]:
(exhibitions
 >> select(X.ExhibitionRole)
 >> mutate(new_ExhibitionRole = X.ExhibitionRole.map(is_artist))
 >> head())

,ExhibitionRole,new_ExhibitionRole
0,Curator,0
1,Artist,1
2,Artist,1
3,Artist,1
4,Artist,1


> I guess the quality of this solution somewhat abides by the DRY principle. One of DRY's successful implementation would have elements that are logically related all change predictably and uniformly, and are thus kept in sync. In the solution above, if you change the defaultdict to a different value, it the new_ExhibitionRole column would change all rows that currently have 0 in them altogether.